In [1]:
import os

import numpy as np
from keras import Sequential, Model
from keras.layers import Dense
from keras.optimizer_v2.adam import Adam

from src.dlla.hw import encode, dlla_hw
from src.dlla.wegener import make_mlp_wegener, binomial_test
from src.pollution.gaussian_noise import gaussian_noise
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed, fixed_fixed
from src.tvla.cri import tvla_cri
from src.tvla.tvla import Group

In [2]:
DB = Database.ascad
TRACE_SET = TraceSetHW(DB)

In [3]:
(X, Y), (X_ATT, Y_ATT) = TRACE_SET.profile(), TRACE_SET.attack()

In [4]:
GAUSS_PARAMS = np.arange(0, 31, 1)

def apply_gauss(params):
    for param in params:
        pollution = Pollution(PollutionType.gauss, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = gaussian_noise(X, param)
            x_att = gaussian_noise(X_ATT, param)

            out.create(x, Y, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Gaussian noise ({param}): min-p: ({min(tvla_p)}).")

apply_gauss(GAUSS_PARAMS)

### TVLA vs. DL-LA

In [5]:
ORDER = 2

def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [ ]:
def prepare_traces_dl(x, y, x_att, y_att):
    """
    Normalizes the traces, one-hot encodes the labels.
    Returns profiling traces, labels and attack traces, labels.
    """
    prof_mean, prof_std = x.mean(axis=0), x.std(axis=0)
    norm_x = (x - prof_mean) / prof_std
    norm_x_att = (x_att - prof_mean) / prof_std

    return norm_x, encode(y), norm_x_att, encode(y_att)


def build_mlp(x, y, params):
    mdl = Sequential()
    mdl.add(Dense(100, activation=params['activation'], input_shape=(x.shape[1],)))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(9, activation='softmax'))

    mdl.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

    out = mdl.fit(x, y, shuffle=True, batch_size=params['batch_size'], epochs=params['epochs'], verbose=False)

    return out, mdl


def make_mlp(x, y):
    return build_mlp(x, y, {
        'activation': 'relu',
        'optimizer': Adam(learning_rate=0.001),
        'losses': 'categorical_crossentropy',
        'batch_size': 150,
        'epochs': 5
    })[1]

def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

def la_gauss(params):
    for param in params:
        pollution = Pollution(PollutionType.gauss, param)
        print("Load traces         ", end="\r")
        trace_set = TraceSetHW(DB, pollution)

        if os.path.exists(trace_set.path):
            print("Prepare traces (1/2)", end="\r")
            x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            print("Prepare traces (2/2)", end="\r")
            (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

            print("Make model (1/2)    ", end="\r")
            mdl9 = make_mlp(x9, y9)
            print("Make model (2/2)    ", end="\r")
            mdl2 = make_mlp_wegener(x2, y2, False)

            print("TVLA                ", end="\r")
            tvla_p = np.min(tvla_cri(trace_set, ORDER)[ORDER])

            print("Predict             ", end="\r")
            dlla9_p, _ = dlla_hw(mdl9, x9_att, y9_att)
            dlla2_p = wegener_p(mdl2, x2_att, y2_att)

            print(f"Gaussian noise ({param}). TVLA_{ORDER} ({tvla_p}). DLLA9 ({dlla9_p}). DLLA2 ({dlla2_p})", end="\r")

            store_results(DB, f"cri_tvla_{ORDER}", pollution, tvla_p)
            store_results(DB, "dlla9", pollution, dlla9_p)
            store_results(DB, "dlla2", pollution, dlla2_p)

        print()

while True:
    la_gauss(GAUSS_PARAMS)

Gaussian noise (0). TVLA_2 (4.213938876300476e-19). DLLA9 (0.0). DLLA2 (0.0)
Gaussian noise (1). TVLA_2 (7.375136678103612e-42). DLLA9 (0.0). DLLA2 (0.0)
Gaussian noise (2). TVLA_2 (3.1549725548904788e-18). DLLA9 (2.4841845786746446e-281). DLLA2 (2.3412774127365286e-222)
Gaussian noise (3). TVLA_2 (7.571723581861694e-13). DLLA9 (3.958562331146552e-176). DLLA2 (1.4347753012276957e-111)
Gaussian noise (4). TVLA_2 (1.1229595537405362e-13). DLLA9 (8.145182955312926e-151). DLLA2 (4.801256227722446e-71)
Gaussian noise (5). TVLA_2 (4.848750128675999e-19). DLLA9 (4.416702388850399e-84). DLLA2 (3.455676267536211e-44)
Gaussian noise (6). TVLA_2 (4.954523969113266e-09). DLLA9 (1.9470478000167267e-35). DLLA2 (9.016066473294937e-51)
Gaussian noise (7). TVLA_2 (6.280371022408742e-11). DLLA9 (6.139956416397127e-08). DLLA2 (5.573417835987549e-29)
Gaussian noise (8). TVLA_2 (4.661060177550371e-08). DLLA9 (0.09105431429073287). DLLA2 (9.804621652738374e-24)
Gaussian noise (9). TVLA_2 (1.1430194432061412